Übung: Volume und mehrere Container in Pod
------------------------------------------------------------------

Diese Übung Demonstriert wie sich zwei Container innerhalb eines Pods das Verzeichnis `/usr/local/apache2/htdocs` teilen.

Der Container `apache` beinhaltet den Web Server und der Container `file-puller` schreibt alle 30 Sekunden die Datei `index.html` in das Verzeichnis `/usr/local/apache2/htdocs`.

Aus Einfachheitsgründen verwenden wir `emptyDir` als Volume.

**Erläuterungen `emptyDir`**

Das emptyDir-Volume wird angelegt, wenn ein Pod einem Node zugewiesen wird.
Alle Container in dem Pod auf diesem (Worker-)Node können dieses emptyDir (einfach
ein leeres Verzeichnis) lesen und schreiben.

Der Pfad, mit dem das emptyDir innerhalb eines Containers eingehängt wird, kann
sich innerhalb der Container des Pods unterscheiden.

Sobald ein Pod von einem Node gelöscht wird, wird auch der Inhalt des emptyDir
komplett und unwiederbringlich gelöscht. Selbst wenn der gleiche Pod auf dem gleichen
Worker Node neu erstellt wird, kann er nicht mehr auf das Volume seines Vorgängers
zugreifen. Dies bezieht sich nicht auf den Crash eines Containers des Pods.

Typische Anwendungsfälle für emptyDir-Volumes könnten sein:
* Laufzeitdaten einer Applikation (Caches), die bei der Neuerstellung des Pods ohnehin
neu erzeugt werden müssten
* Übergabe von (Laufzeit-)Konfigurationsdaten an alle Container innerhalb des
Pods

Zuerst schauen wir den Inhalt der YAML Datei an und starten dann die Ressourcen

In [34]:
! cat 09-5-Volume/web.yaml

apiVersion: v1
kind: Pod
metadata:
  labels:
    app.kubernetes.io/name: web
  name: web
spec:
  containers:
  - image: httpd
    name: apache
    # Volumes im Container
    volumeMounts:
    - mountPath: "/usr/local/apache2/htdocs"
      name: "web-storage"    
  - image: debian:jessie
    name: file-puller
    # Just spin & wait forever
    command: [ "/bin/bash", "-c", "--" ]
    args: [ "while true; do echo \"<html><body><h1>Hallo es ist $(date)</h1></body></html>\" >/usr/local/apache2/htdocs/index.html; sleep 30; done;" ]    
    # Volumes im Container
    volumeMounts:
    - mountPath: "/usr/local/apache2/htdocs"
      name: "web-storage"
  # Volumes in Host      
  volumes:
  - name: "web-storage"
    emptyDir: {}   
---
apiVersion: v1
kind: Service
metadata:
  labels:
    app.kubernetes.io/name: web
  name: web
spec:
  ports:
  - port: 80
    protocol: TCP
    targetPort: 80
  selector:
    app.kubernetes.io/name: web
  type: LoadBalancer


In [35]:
! kubectl apply -f 09-5-Volume/web.yaml

pod/web created
service/web created


Um das Ergebnis anschauen, holen wir uns die IP-Adresse des Loadbalancer und des Services

In [37]:
! kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service web -o=jsonpath='{ .spec.ports[0].nodePort }')/"

http://192.168.178.200:31371

Und geben die Erzeugten Ressourcen aus:

In [42]:
! kubectl get pods,services
! echo "\n\nContainer"
! kubectl get pods -o=jsonpath='{range .items[*]}{"\n"}{.metadata.name}{":\t"}{range .spec.containers[*]}{.image}{", "}{end}{end}' |\
sort

NAME                                READY   STATUS    RESTARTS   AGE
pod/jupyter-base-57c55b7fc8-d9l85   1/1     Running   0          2d15h
pod/nginxpod1                       1/1     Running   0          2d15h
pod/web                             2/2     Running   0          3m24s

NAME                   TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
service/jupyter-base   LoadBalancer   10.109.197.23   <pending>     8888:32188/TCP   2d15h
service/kubernetes     ClusterIP      10.96.0.1       <none>        443/TCP          8d
service/web            LoadBalancer   10.98.166.25    <pending>     80:31371/TCP     3m24s


Container

jupyter-base-57c55b7fc8-d9l85:	jupyter/base-notebook, 
nginxpod1:	nginx, 
web:	httpd, debian:jessie, 


Aufräumen

In [43]:
! kubectl delete -f 09-5-Volume/web.yaml

pod "web" deleted
service "web" deleted
